In [1]:
from model import get_model
import tensorflow as tf
from tensorflow.keras.layers import Lambda, Reshape
from tensorflow.keras.models import Sequential

In [2]:
def preprocess_image_layer():
    def single_img_process_tf(img):
        img = tf.image.rgb_to_grayscale(img)

        original_shape = tf.shape(img)
        original_height = original_shape[1]
        original_width = original_shape[2]
        new_width = original_width * 32 // original_height

        img = tf.image.resize(img, [32, new_width])
        img = tf.cast(img, tf.float32)

        img = img / 255.0 - 0.5

        img = tf.expand_dims(img, -1)

        return img

    return Lambda(single_img_process_tf, name='preprocessing_layer')

# Load your existing model
predict_model, train_model = get_model()

# Create a new model that includes the preprocessing layer
model_with_preprocessing = Sequential([
    preprocess_image_layer(),
    predict_model
])

In [3]:
model_with_preprocessing.build((None, None, None, 3))
model_with_preprocessing.summary()

tf.saved_model.save(model_with_preprocessing, 'save')

# 转化为tflite
# converter = tf.lite.TFLiteConverter.from_keras_model(model_with_preprocessing)
# tflite_model = converter.convert()

converter = tf.lite.TFLiteConverter.from_saved_model('save')
converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TFLite ops
        tf.lite.OpsSet.SELECT_TF_OPS,  # enable TF ops
    ]
tflite_model = converter.convert()

with open('save_model.tflite', 'wb') as f:
    f.write(tflite_model)

#保存tflite
# with open('densenet.tflite', 'wb') as f:
#     f.write(tflite_model)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocessing_layer (Lambda  (None, 32, None, 1, 1)   0         
 )                                                               
                                                                 
 model (Functional)          (None, None, 5991)        4897511   
                                                                 
Total params: 4,897,511
Trainable params: 4,890,023
Non-trainable params: 7,488
_________________________________________________________________


INFO:tensorflow:Assets written to: save\assets


INFO:tensorflow:Assets written to: save\assets


In [4]:
import os
model_path = os.path.join(os.getcwd(), 'save_model.tflite')
print('TF Lite model:', model_path)

TF Lite model: D:\ProgramData\IO\mt-io\ocr\densenet\save_model.tflite


In [5]:
input_image = tf.random.normal(shape=(1, 200, 500, 3), dtype=tf.float32)
# Run inference with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_path=model_path)

input_details = interpreter.get_input_details()
interpreter.resize_tensor_input(input_details[0]['index'], (1, 200, 500, 3), strict=True)
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]["index"], input_image)

print(input_details)

interpreter.invoke()
output = interpreter.tensor(interpreter.get_output_details()[0]["index"])()[0]
print(output.shape)
print(output)

[{'name': 'serving_default_preprocessing_layer_input:0', 'index': 0, 'shape': array([1, 1, 1, 3]), 'shape_signature': array([-1, -1, -1,  3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
(10, 5991)
[[0.00019616 0.00019973 0.00018511 ... 0.00017426 0.0002028  0.0001398 ]
 [0.00019408 0.00021143 0.00017693 ... 0.00016959 0.00018444 0.00013365]
 [0.00019141 0.00020878 0.00017289 ... 0.00016279 0.00018847 0.00014025]
 ...
 [0.00019257 0.00020875 0.00017403 ... 0.00016126 0.00018833 0.00014149]
 [0.00019874 0.00020396 0.00017802 ... 0.00016521 0.00018433 0.00014263]
 [0.00020595 0.00021229 0.00016106 ... 0.00015685 0.00017544 0.00014268]]
